In [1]:
from itertools import product
from math import sin, cos
import numpy as np
from z3 import *

In [2]:
V_STOP, V_LOW, V_HIGH = 0, 1, 10 # velocidades dos modos baixo e alto, em m
L = 1e3 # Lado dos setores em m
THETA = 15 # angulo de viragem entre modos
TAU = 3 # tempo minimo entre transicoes timed
ALL_ROUTES = [i*THETA for i in range(int(360/THETA))] # todos os ângulos possíveis
ALL_VELS = [V_STOP, V_LOW, V_HIGH] # todas as velocidades possiveis

Mode, (M_STOP, M_LOW, M_HIGH) = EnumSort("Mode", ("V_STOP", "V_LOW", "V_HIGH"))

In [3]:
deg_to_rad = lambda a: a * np.pi / 180
vel_to_mode = lambda v: M_LOW if (v == V_LOW) else M_HIGH if (v == V_HIGH) else M_STOP
mode_to_vel = lambda m: V_LOW if (m == M_LOW) else V_HIGH if (m == M_HIGH) else V_STOP

In [4]:
def declare(i, num_boats=3):
    trace = {}
    for j in range(num_boats):
        trace[f"b{j}"] = {}
        trace[f"b{j}"]["v"] = Const(f"b{j}_v{i}", Mode)
        trace[f"b{j}"]["a"] = Int(f"b{j}_a{i}")
        trace[f"b{j}"]["sx"] = Int(f"b{j}_sx{i}")
        trace[f"b{j}"]["sy"] = Int(f"b{j}_sy{i}")
        trace[f"b{j}"]["x"] = Real(f"b{j}_x{i}")
        trace[f"b{j}"]["y"] = Real(f"b{j}_y{i}")
        trace[f"b{j}"]["t"] = Real(f"b{j}_t{i}")
    
    return trace
        
        
def init(trace, N):
    r = []
    for b in trace:
        if "b" in b:
            r.append(trace[b]["v"] == M_HIGH)
            index = np.random.randint(len(ALL_ROUTES))
            r.append(trace[b]["a"] == ALL_ROUTES[index])
            r.append(trace[b]["sx"] == np.random.randint(N))
            r.append(trace[b]["sy"] == np.random.randint(N))
            r.append(And(trace[b]["x"] >= L*trace[b]["sx"], trace[b]["x"] <= L*(trace[b]["sx"]+1)))
            r.append(And(trace[b]["y"] >= L*trace[b]["sy"], trace[b]["y"] <= L*(trace[b]["sy"]+1)))
            r.append(trace[b]["t"] == 0)
            
    return And(r)

In [5]:
def update_pos(prev, curr, b, dt):
    r = []
    modes = list(product(*[ALL_ROUTES, ALL_VELS]))
    
    for v, a in modes:
        vel = prev[b]["v"] == vel_to_mode(v)
        ang = prev[b]["a"] == a
        x = curr[b]["x"] == prev[b]["x"] + v * cos(deg_to_rad(a)) * dt
        y = curr[b]["y"] == prev[b]["y"] + v * sin(deg_to_rad(a)) * dt
        r.append(And(x, y, ang, vel))
    
    return Or(r)

def high_low(prev, curr, b):
    r = []
    
    # TODO: adicionar resticoes a rota
    r.append(And(prev[b]["v"] == M_HIGH, curr[b]["v"] == M_LOW))
    r.append(curr[b]["t"] == prev[b]["t"] + 500)
    r.append(update_pos(prev, curr, b, 500))
    
    return And(r)
    
def low_high(prev, curr, b):
    r = []
    
    # TODO: adicionar resticoes a rota
    r.append(And(prev[b]["v"] == M_LOW, curr[b]["v"] == M_HIGH))
    r.append(curr[b]["t"] == prev[b]["t"] + 500)
    r.append(update_pos(prev, curr, b, 500))
    
    return And(r)
    
def low_stop(prev, curr, b):
    r = []
    
    # TODO: adicionar resticoes a rota
    r.append(And(prev[b]["v"] == M_LOW, curr[b]["v"] == M_STOP))
    r.append(curr[b]["t"] == prev[b]["t"] + 50)
    r.append(update_pos(prev, curr, b, 50))
    
    return And(r)
    
def stop_low(prev, curr, b):
    r = []
    
    # TODO: adicionar resticoes a rota
    r.append(And(prev[b]["v"] == M_STOP, curr[b]["v"] == M_LOW))
    r.append(curr[b]["t"] == prev[b]["t"] + 50)
    r.append(update_pos(prev, curr, b, 50))
    
    return And(r)

In [6]:
trace, trace1 = declare(0), declare(1)
conds = init(trace, 10)
print(high_low(trace, trace1, "b1"))

And(And(b1_v0 == V_HIGH, b1_v1 == V_LOW),
    b1_t1 == b1_t0 + 500,
    Or(And(b1_x1 == b1_x0 + 0,
           b1_y1 == b1_y0 + 0,
           b1_a0 == 0,
           b1_v0 == V_STOP),
       And(b1_x1 == b1_x0 + 0,
           b1_y1 == b1_y0 + 0,
           b1_a0 == 1,
           b1_v0 == V_STOP),
       And(b1_x1 == b1_x0 + 0,
           b1_y1 == b1_y0 + 0,
           b1_a0 == 10,
           b1_v0 == V_STOP),
       And(b1_x1 == b1_x0 + 7500,
           b1_y1 == b1_y0 + 0,
           b1_a0 == 0,
           b1_v0 == V_STOP),
       And(b1_x1 == b1_x0 + 1499771542734587/200000000000,
           b1_y1 ==
           b1_y0 + 13089304827962633/100000000000000,
           b1_a0 == 1,
           b1_v0 == V_STOP),
       And(b1_x1 == b1_x0 + 7386058147591561/1000000000000,
           b1_y1 == b1_y0 + 6511806662509887/5000000000000,
           b1_a0 == 10,
           b1_v0 == V_STOP),
       And(b1_x1 == b1_x0 + 15000,
           b1_y1 == b1_y0 + 0,
           b1_a0 == 0,
           b1_v0 == V_STO

In [7]:
mode_to_vel(M_LOW)

1